In [1]:
import numpy as np 
import pandas as pd 
# import necessary libraries for training Electra 
from transformers import * 
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
import random 
import time 
import datetime 
from tqdm import tqdm 
import gc 
import matplotlib.pyplot as plt 
import seaborn as sns 
# import necessary libraries for training LSTM 
import tensorflow as tf 
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import re 

## Load Dataset

In [74]:
# import dataframes from the 2018 research 
# train1 contains inconsistent news 
# train2 contains irrelevant news  
# we are not using mismatch news yet.  
train1 = pd.read_csv('mission1_train.csv')
train2 = pd.read_csv('mission2_train.csv')

train = pd.concat([train1,train2], axis = 0)
train = train.dropna() 
train = train.drop(columns=['seqid'])

train['title_length'] = train['title'].apply(lambda x : len(x)) 
train = train[train['title_length'] <= 120] 
train = train[train['title_length'] >= 5] 

print(train.shape)

(98093, 4)


In [75]:
test = pd.read_csv('mindslab_test.csv') 

In [78]:
test.shape

(200, 3)

## Segment data using sliding window method

In [79]:
def clean_text(s): 
    FILTERS = "([~.,!?\"':;(])"
    CHANGE_FILTER = re.compile(FILTERS)
    return re.sub(CHANGE_FILTER, " ", s) 

In [38]:
def split_text(s, overlap = 20, chunk_size = 50): 
    total = [] 
    partial = [] 
    if len(s.split()) // (chunk_size - overlap) > 0:  
        n = len(s.split()) // (chunk_size - overlap) 
    else: 
        n = 1 
    for w in range(n): 
        if w == 0: 
            partial = s.split()[:chunk_size] 
            total.append(" ".join(partial)) 
        else:  
            partial = s.split()[w*(chunk_size - overlap):w*(chunk_size - overlap) + chunk_size]
            total.append(" ".join(partial)) 
    return total 

In [39]:
contents = train['content'].values
titles = train['title'].values 
labels = train['Label'].values 

In [40]:
data = {'title':[], 'content':[], 'Label':[]} 
for i in range(len(contents)): 
    splitted = split_text(clean_text(contents[i]))
    for text in splitted: 
        data['title'].append(titles[i]) 
        data['content'].append(text) 
        data['Label'].append(labels[i]) 

In [41]:
splitted_train = pd.DataFrame(data) 

In [60]:
splitted_train.head()

,title,content,Label
0,"박상기 법무장관 후보자, 자동차세·과태료 미납으로 15차례 차량 압류",박상기 법무부장관 후보자가 교통법규 위반으로 부과된 과태료를 7차례 체납하고 자동차...,0.0
1,"박상기 법무장관 후보자, 자동차세·과태료 미납으로 15차례 차량 압류",해서 후보자 차량 압류가 오랜 기간 이뤄졌다’고 지적했다 주 의원에 따르면 박 후보...,0.0
2,"박상기 법무장관 후보자, 자동차세·과태료 미납으로 15차례 차량 압류",2015년 6월 8일 해당 과태료를 납부했다 2011년 4월 22일 부과된 ‘속도위...,0.0
3,"박상기 법무장관 후보자, 자동차세·과태료 미납으로 15차례 차량 압류",또 자동차세와 과태료 미납으로 차량 압류를 15차례 당했던 것으로 확인됐다 2008...,0.0
4,"박상기 법무장관 후보자, 자동차세·과태료 미납으로 15차례 차량 압류",압류 기록은 작년 10월 6일에 있었다 박 후보자는 이에 대해 청문회에서 “저는 대...,0.0


## Tokenize Data

In [48]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [61]:
def electra_tokenizer_simple(sent1, sent2, MAX_LEN):  
    encoded_dict = tokenizer.encode_plus(
        text = sent1, 
        text_pair = sent2,  
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences    
    
    if len(input_id) > 512: 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        token_type_id = token_type_id[:129] + token_type_id[-383:]   
    elif len(input_id) < 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        token_type_id = token_type_id + [0]*(512 - len(token_type_id))  
        
    return np.asarray(input_id), np.asarray(attention_mask), np.asarray(token_type_id) 

In [62]:
MAX_LEN = 512
input_ids = []
attention_masks = [] 
token_type_ids = [] 
cnt = 0
for sent1, sent2 in zip(splitted_train['title'], splitted_train['content']): 
    if cnt%1000 == 0 and cnt > 0: 
        print("Processed {} datapoints".format(cnt)) 
    cnt += 1
    try: 
        input_id, attention_mask, token_type_id = electra_tokenizer_simple(sent1, sent2, MAX_LEN)
        ## check if the number of tokens exceed 510 (excluding [cls] and [sep]) 
        ## if so empirically select the first 129 and the last 383 tokens 
        input_ids.append(input_id) 
        attention_masks.append(attention_mask) 
        token_type_ids.append(token_type_id) 
    except Exception as e:  
        print(e)         
        print(sent1, sent2) 
        pass

Processed 1000 datapoints
Processed 2000 datapoints
Processed 3000 datapoints
Processed 4000 datapoints
Processed 5000 datapoints
Processed 6000 datapoints
Processed 7000 datapoints
Processed 8000 datapoints
Processed 9000 datapoints
Processed 10000 datapoints
Processed 11000 datapoints
Processed 12000 datapoints
Processed 13000 datapoints
Processed 14000 datapoints
Processed 15000 datapoints
Processed 16000 datapoints
Processed 17000 datapoints
Processed 18000 datapoints
Processed 19000 datapoints
Processed 20000 datapoints
Processed 21000 datapoints
Processed 22000 datapoints
Processed 23000 datapoints
Processed 24000 datapoints
Processed 25000 datapoints
Processed 26000 datapoints
Processed 27000 datapoints
Processed 28000 datapoints
Processed 29000 datapoints
Processed 30000 datapoints
Processed 31000 datapoints
Processed 32000 datapoints
Processed 33000 datapoints
Processed 34000 datapoints
Processed 35000 datapoints
Processed 36000 datapoints
Processed 37000 datapoints
Processed 

Processed 298000 datapoints
Processed 299000 datapoints
Processed 300000 datapoints
Processed 301000 datapoints
Processed 302000 datapoints
Processed 303000 datapoints
Processed 304000 datapoints
Processed 305000 datapoints
Processed 306000 datapoints
Processed 307000 datapoints
Processed 308000 datapoints
Processed 309000 datapoints
Processed 310000 datapoints
Processed 311000 datapoints
Processed 312000 datapoints
Processed 313000 datapoints
Processed 314000 datapoints
Processed 315000 datapoints
Processed 316000 datapoints
Processed 317000 datapoints
Processed 318000 datapoints
Processed 319000 datapoints
Processed 320000 datapoints
Processed 321000 datapoints
Processed 322000 datapoints
Processed 323000 datapoints
Processed 324000 datapoints
Processed 325000 datapoints
Processed 326000 datapoints
Processed 327000 datapoints
Processed 328000 datapoints
Processed 329000 datapoints
Processed 330000 datapoints
Processed 331000 datapoints
Processed 332000 datapoints
Processed 333000 dat

Processed 591000 datapoints
Processed 592000 datapoints
Processed 593000 datapoints
Processed 594000 datapoints
Processed 595000 datapoints
Processed 596000 datapoints
Processed 597000 datapoints
Processed 598000 datapoints
Processed 599000 datapoints
Processed 600000 datapoints
Processed 601000 datapoints
Processed 602000 datapoints
Processed 603000 datapoints
Processed 604000 datapoints
Processed 605000 datapoints
Processed 606000 datapoints
Processed 607000 datapoints
Processed 608000 datapoints
Processed 609000 datapoints
Processed 610000 datapoints
Processed 611000 datapoints
Processed 612000 datapoints
Processed 613000 datapoints
Processed 614000 datapoints
Processed 615000 datapoints
Processed 616000 datapoints
Processed 617000 datapoints
Processed 618000 datapoints
Processed 619000 datapoints
Processed 620000 datapoints
Processed 621000 datapoints
Processed 622000 datapoints
Processed 623000 datapoints
Processed 624000 datapoints
Processed 625000 datapoints
Processed 626000 dat

In [63]:
input_ids = np.asarray(input_ids)
attention_masks = np.asarray(attention_masks) 
token_type_ids = np.asarray(token_type_ids)
y_train = splitted_train['Label'].values 

In [64]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, y_train, random_state = 2021, test_size = 0.1, stratify = y_train)

train_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, random_state = 2021, test_size = 0.1)

train_token_type_ids, val_token_type_ids, _, _ = train_test_split(token_type_ids, input_ids, random_state = 2021, test_size = 0.1)


In [65]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_type_ids = torch.tensor(train_token_type_ids) 

In [66]:
val_inputs = torch.tensor(val_inputs) 
val_labels = torch.tensor(val_labels) 
val_masks = torch.tensor(val_masks) 
val_token_type_ids = torch.tensor(val_token_type_ids)

In [67]:
batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_token_type_ids, train_labels) 
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size) 

In [68]:
val_data = TensorDataset(val_inputs, val_masks, val_token_type_ids, val_labels) 
val_sampler = SequentialSampler(val_data) 
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size = batch_size)  

## Define Model

In [69]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator")   
model.cuda() 

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [70]:
def format_time(elapsed): 
    elapsed_rounded = int(round(elapsed)) 
    return str(datetime.timedelta(seconds = elapsed_rounded))

In [71]:
predictions = [] 
def compute_accuracy(model, dataloader, device):
    tqdm()
    model.eval()
    correct_preds, num_samples = 0,0
    with torch.no_grad():
        for i, batch in enumerate(tqdm(dataloader)):
            b_input_ids, b_input_masks, b_token_type_ids, b_labels = tuple(t.to(device) for t in batch) 
            loss, yhat = model(input_ids=b_input_ids, attention_mask=b_input_masks, 
                               token_type_ids = b_token_type_ids, labels=b_labels.long())
            prediction = (torch.sigmoid(yhat[:,1]) > 0.5).long() 
            predictions.append(prediction)
            num_samples += b_labels.size(0)
            correct_preds += (prediction==b_labels.long()).sum()
            del b_input_ids, b_input_masks, b_token_type_ids, b_labels #memory
        torch.cuda.empty_cache() #memory
        gc.collect() # memory 
        return correct_preds.float()/num_samples*100 


In [72]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
device

device(type='cuda', index=0)

In [73]:
seed_val = 8888 
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8) 
epochs = 10 # 10 epochs for now. We may transfer learn later 
total_steps = len(train_dataloader) * epochs 
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps) 

losses = [] 
val_losses = [] 
#model.zero_grad() 

for epoch_i in range(0, epochs): 
    print("")
    print("===== Epoch {:} / {:} =====".format(epoch_i + 1, epochs)) 
    print("Training ...") 
    t0 = time.time()
    running_loss = 0 
    train_accuracy = 0
    iteration = 0 
    model.train() 
    for step, batch in enumerate(train_dataloader):
        iteration += 1 
        batch = tuple(t.to(device) for t in batch) 
        b_input_ids, b_input_masks, b_token_type_ids, b_labels = batch 
        optimizer.zero_grad()
        loss, yhat = model(b_input_ids, attention_mask = b_input_masks, 
                        token_type_ids = b_token_type_ids, labels = b_labels.long()) 

        loss.backward()
        optimizer.step()
        scheduler.step()  

        running_loss += float(loss.item())
        del b_input_ids, b_input_masks, b_token_type_ids, b_labels #memory
        
        if not step%25:
            print(f'Epoch: {epoch_i+1:03d}/{epochs:03d} | '
                  f'Batch {step+1:03d}/{len(train_dataloader):03d} | '
                  f'Average Loss in last {iteration} iteration(s): {(running_loss/iteration):.4f} | '
                  f'Elapsed {format_time(time.time()-t0)}')
            running_loss = 0.0
            iteration = 0
        torch.cuda.empty_cache() #memory
        gc.collect() #memory
        losses.append(float(loss.item()))
    
    with torch.no_grad():
        print(f'\nTraining Accuracy: ' f'{compute_accuracy(model, train_dataloader, device):.2f}%')
        
        
    print("Calculating validation metrics...")
    model.eval() 
    eval_loss = 0
    eval_accuracy = 0
    for batch in val_dataloader: 
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_masks, b_token_type_ids, b_labels = batch 
        with torch.no_grad(): 
            loss, yhat = model(b_input_ids, attention_mask = b_input_masks, 
                        token_type_ids = b_token_type_ids, labels = b_labels.long())  
        eval_loss += float(loss.item())  
        del b_input_ids, b_input_masks, b_token_type_ids, b_labels # memory 
        val_losses.append(float(loss.item())) 
        
    avg_val_loss = eval_loss / len(val_dataloader) 
    # avg_val_accuracy = eval_accuracy / len(val_dataloader)   
    
    print("Average validation loss = {}".format(avg_val_loss)) 
    print("Average validation accuracy = {}".format(compute_accuracy(model, val_dataloader, device)))  
    
    # saving model 
    torch.save(model.state_dict(), 'electra_chunked_ver_2_' + str(epoch_i + 1))



===== Epoch 1 / 10 =====
Training ...
Epoch: 001/010 | Batch 001/45709 | Average Loss in last 1 iteration(s): 0.6811 | Elapsed 0:00:01
Epoch: 001/010 | Batch 026/45709 | Average Loss in last 25 iteration(s): 0.6957 | Elapsed 0:00:27
Epoch: 001/010 | Batch 051/45709 | Average Loss in last 25 iteration(s): 0.6914 | Elapsed 0:00:52
Epoch: 001/010 | Batch 076/45709 | Average Loss in last 25 iteration(s): 0.6942 | Elapsed 0:01:21
Epoch: 001/010 | Batch 101/45709 | Average Loss in last 25 iteration(s): 0.6903 | Elapsed 0:01:45
Epoch: 001/010 | Batch 126/45709 | Average Loss in last 25 iteration(s): 0.6901 | Elapsed 0:02:11
Epoch: 001/010 | Batch 151/45709 | Average Loss in last 25 iteration(s): 0.6810 | Elapsed 0:02:37
Epoch: 001/010 | Batch 176/45709 | Average Loss in last 25 iteration(s): 0.6581 | Elapsed 0:03:03
Epoch: 001/010 | Batch 201/45709 | Average Loss in last 25 iteration(s): 0.6332 | Elapsed 0:03:25
Epoch: 001/010 | Batch 226/45709 | Average Loss in last 25 iteration(s): 0.6422 

Epoch: 001/010 | Batch 2076/45709 | Average Loss in last 25 iteration(s): 0.4215 | Elapsed 0:35:49
Epoch: 001/010 | Batch 2101/45709 | Average Loss in last 25 iteration(s): 0.4809 | Elapsed 0:36:13
Epoch: 001/010 | Batch 2126/45709 | Average Loss in last 25 iteration(s): 0.4456 | Elapsed 0:36:40
Epoch: 001/010 | Batch 2151/45709 | Average Loss in last 25 iteration(s): 0.4777 | Elapsed 0:37:06
Epoch: 001/010 | Batch 2176/45709 | Average Loss in last 25 iteration(s): 0.4547 | Elapsed 0:37:32
Epoch: 001/010 | Batch 2201/45709 | Average Loss in last 25 iteration(s): 0.4735 | Elapsed 0:37:55
Epoch: 001/010 | Batch 2226/45709 | Average Loss in last 25 iteration(s): 0.4646 | Elapsed 0:38:23
Epoch: 001/010 | Batch 2251/45709 | Average Loss in last 25 iteration(s): 0.4722 | Elapsed 0:38:50
Epoch: 001/010 | Batch 2276/45709 | Average Loss in last 25 iteration(s): 0.4739 | Elapsed 0:39:17
Epoch: 001/010 | Batch 2301/45709 | Average Loss in last 25 iteration(s): 0.4378 | Elapsed 0:39:43
Epoch: 001

Epoch: 001/010 | Batch 4151/45709 | Average Loss in last 25 iteration(s): 0.4340 | Elapsed 1:11:14
Epoch: 001/010 | Batch 4176/45709 | Average Loss in last 25 iteration(s): 0.4702 | Elapsed 1:11:44
Epoch: 001/010 | Batch 4201/45709 | Average Loss in last 25 iteration(s): 0.4192 | Elapsed 1:12:08
Epoch: 001/010 | Batch 4226/45709 | Average Loss in last 25 iteration(s): 0.4261 | Elapsed 1:12:35
Epoch: 001/010 | Batch 4251/45709 | Average Loss in last 25 iteration(s): 0.4647 | Elapsed 1:13:00
Epoch: 001/010 | Batch 4276/45709 | Average Loss in last 25 iteration(s): 0.4102 | Elapsed 1:13:26
Epoch: 001/010 | Batch 4301/45709 | Average Loss in last 25 iteration(s): 0.4497 | Elapsed 1:13:48
Epoch: 001/010 | Batch 4326/45709 | Average Loss in last 25 iteration(s): 0.4548 | Elapsed 1:14:14
Epoch: 001/010 | Batch 4351/45709 | Average Loss in last 25 iteration(s): 0.4321 | Elapsed 1:14:43
Epoch: 001/010 | Batch 4376/45709 | Average Loss in last 25 iteration(s): 0.3959 | Elapsed 1:15:08
Epoch: 001

Epoch: 001/010 | Batch 6226/45709 | Average Loss in last 25 iteration(s): 0.3738 | Elapsed 1:47:06
Epoch: 001/010 | Batch 6251/45709 | Average Loss in last 25 iteration(s): 0.4278 | Elapsed 1:47:32
Epoch: 001/010 | Batch 6276/45709 | Average Loss in last 25 iteration(s): 0.3936 | Elapsed 1:48:00
Epoch: 001/010 | Batch 6301/45709 | Average Loss in last 25 iteration(s): 0.4023 | Elapsed 1:48:25
Epoch: 001/010 | Batch 6326/45709 | Average Loss in last 25 iteration(s): 0.3347 | Elapsed 1:48:50
Epoch: 001/010 | Batch 6351/45709 | Average Loss in last 25 iteration(s): 0.3689 | Elapsed 1:49:13
Epoch: 001/010 | Batch 6376/45709 | Average Loss in last 25 iteration(s): 0.3909 | Elapsed 1:49:42
Epoch: 001/010 | Batch 6401/45709 | Average Loss in last 25 iteration(s): 0.4338 | Elapsed 1:50:09
Epoch: 001/010 | Batch 6426/45709 | Average Loss in last 25 iteration(s): 0.3315 | Elapsed 1:50:35
Epoch: 001/010 | Batch 6451/45709 | Average Loss in last 25 iteration(s): 0.3833 | Elapsed 1:51:02
Epoch: 001

Epoch: 001/010 | Batch 8301/45709 | Average Loss in last 25 iteration(s): 0.3578 | Elapsed 2:23:04
Epoch: 001/010 | Batch 8326/45709 | Average Loss in last 25 iteration(s): 0.3059 | Elapsed 2:23:32
Epoch: 001/010 | Batch 8351/45709 | Average Loss in last 25 iteration(s): 0.3736 | Elapsed 2:23:57
Epoch: 001/010 | Batch 8376/45709 | Average Loss in last 25 iteration(s): 0.3586 | Elapsed 2:24:21
Epoch: 001/010 | Batch 8401/45709 | Average Loss in last 25 iteration(s): 0.3865 | Elapsed 2:24:50
Epoch: 001/010 | Batch 8426/45709 | Average Loss in last 25 iteration(s): 0.3553 | Elapsed 2:25:16
Epoch: 001/010 | Batch 8451/45709 | Average Loss in last 25 iteration(s): 0.3268 | Elapsed 2:25:42
Epoch: 001/010 | Batch 8476/45709 | Average Loss in last 25 iteration(s): 0.3819 | Elapsed 2:26:09
Epoch: 001/010 | Batch 8501/45709 | Average Loss in last 25 iteration(s): 0.3648 | Elapsed 2:26:33
Epoch: 001/010 | Batch 8526/45709 | Average Loss in last 25 iteration(s): 0.3608 | Elapsed 2:26:56
Epoch: 001

...

Epoch: 003/010 | Batch 18651/45709 | Average Loss in last 25 iteration(s): 0.1059 | Elapsed 5:35:00
Epoch: 003/010 | Batch 18676/45709 | Average Loss in last 25 iteration(s): 0.1069 | Elapsed 5:35:27
Epoch: 003/010 | Batch 18701/45709 | Average Loss in last 25 iteration(s): 0.0710 | Elapsed 5:35:55
Epoch: 003/010 | Batch 18726/45709 | Average Loss in last 25 iteration(s): 0.0689 | Elapsed 5:36:20
Epoch: 003/010 | Batch 18751/45709 | Average Loss in last 25 iteration(s): 0.0613 | Elapsed 5:36:45
Epoch: 003/010 | Batch 18776/45709 | Average Loss in last 25 iteration(s): 0.0974 | Elapsed 5:37:17
Epoch: 003/010 | Batch 18801/45709 | Average Loss in last 25 iteration(s): 0.0596 | Elapsed 5:37:42
Epoch: 003/010 | Batch 18826/45709 | Average Loss in last 25 iteration(s): 0.0537 | Elapsed 5:38:11
Epoch: 003/010 | Batch 18851/45709 | Average Loss in last 25 iteration(s): 0.0801 | Elapsed 5:38:37
Epoch: 003/010 | Batch 18876/45709 | Average Loss in last 25 iteration(s): 0.0723 | Elapsed 5:39:03


Epoch: 003/010 | Batch 20701/45709 | Average Loss in last 25 iteration(s): 0.0928 | Elapsed 6:11:47
Epoch: 003/010 | Batch 20726/45709 | Average Loss in last 25 iteration(s): 0.0366 | Elapsed 6:12:12
Epoch: 003/010 | Batch 20751/45709 | Average Loss in last 25 iteration(s): 0.0812 | Elapsed 6:12:41
Epoch: 003/010 | Batch 20776/45709 | Average Loss in last 25 iteration(s): 0.0766 | Elapsed 6:13:10
Epoch: 003/010 | Batch 20801/45709 | Average Loss in last 25 iteration(s): 0.0494 | Elapsed 6:13:35
Epoch: 003/010 | Batch 20826/45709 | Average Loss in last 25 iteration(s): 0.0650 | Elapsed 6:14:01
Epoch: 003/010 | Batch 20851/45709 | Average Loss in last 25 iteration(s): 0.0671 | Elapsed 6:14:27
Epoch: 003/010 | Batch 20876/45709 | Average Loss in last 25 iteration(s): 0.0717 | Elapsed 6:14:52
Epoch: 003/010 | Batch 20901/45709 | Average Loss in last 25 iteration(s): 0.0511 | Elapsed 6:15:19
Epoch: 003/010 | Batch 20926/45709 | Average Loss in last 25 iteration(s): 0.0624 | Elapsed 6:15:49


Epoch: 003/010 | Batch 22751/45709 | Average Loss in last 25 iteration(s): 0.0885 | Elapsed 6:48:40
Epoch: 003/010 | Batch 22776/45709 | Average Loss in last 25 iteration(s): 0.0793 | Elapsed 6:49:06
Epoch: 003/010 | Batch 22801/45709 | Average Loss in last 25 iteration(s): 0.0581 | Elapsed 6:49:34
Epoch: 003/010 | Batch 22826/45709 | Average Loss in last 25 iteration(s): 0.0697 | Elapsed 6:50:00
Epoch: 003/010 | Batch 22851/45709 | Average Loss in last 25 iteration(s): 0.0627 | Elapsed 6:50:24
Epoch: 003/010 | Batch 22876/45709 | Average Loss in last 25 iteration(s): 0.0647 | Elapsed 6:50:52
Epoch: 003/010 | Batch 22901/45709 | Average Loss in last 25 iteration(s): 0.0666 | Elapsed 6:51:20
Epoch: 003/010 | Batch 22926/45709 | Average Loss in last 25 iteration(s): 0.0720 | Elapsed 6:51:48
Epoch: 003/010 | Batch 22951/45709 | Average Loss in last 25 iteration(s): 0.0889 | Elapsed 6:52:15
Epoch: 003/010 | Batch 22976/45709 | Average Loss in last 25 iteration(s): 0.0703 | Elapsed 6:52:41


Epoch: 003/010 | Batch 24801/45709 | Average Loss in last 25 iteration(s): 0.0783 | Elapsed 7:25:33
Epoch: 003/010 | Batch 24826/45709 | Average Loss in last 25 iteration(s): 0.0502 | Elapsed 7:25:56
Epoch: 003/010 | Batch 24851/45709 | Average Loss in last 25 iteration(s): 0.0531 | Elapsed 7:26:22
Epoch: 003/010 | Batch 24876/45709 | Average Loss in last 25 iteration(s): 0.0557 | Elapsed 7:26:53
Epoch: 003/010 | Batch 24901/45709 | Average Loss in last 25 iteration(s): 0.1004 | Elapsed 7:27:19
Epoch: 003/010 | Batch 24926/45709 | Average Loss in last 25 iteration(s): 0.0697 | Elapsed 7:27:47
Epoch: 003/010 | Batch 24951/45709 | Average Loss in last 25 iteration(s): 0.0972 | Elapsed 7:28:15
Epoch: 003/010 | Batch 24976/45709 | Average Loss in last 25 iteration(s): 0.0731 | Elapsed 7:28:40
Epoch: 003/010 | Batch 25001/45709 | Average Loss in last 25 iteration(s): 0.0776 | Elapsed 7:29:08
Epoch: 003/010 | Batch 25026/45709 | Average Loss in last 25 iteration(s): 0.0360 | Elapsed 7:29:36


KeyboardInterrupt: 